В этом ноутбуке загружал тренировочные данные по 7 фолдам.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/alfa-bank-train-target-folds/train_target_fold_6.csv
/kaggle/input/alfa-bank-train-target-folds/train_target_fold_2.csv
/kaggle/input/alfa-bank-train-target-folds/val_target_fold_3.csv
/kaggle/input/alfa-bank-train-target-folds/train_target_fold_7.csv
/kaggle/input/alfa-bank-train-target-folds/val_target_fold_6.csv
/kaggle/input/alfa-bank-train-target-folds/val_target_fold_2.csv
/kaggle/input/alfa-bank-train-target-folds/embedding_projections_dictionary .pkl
/kaggle/input/alfa-bank-train-target-folds/val_target_fold_4.csv
/kaggle/input/alfa-bank-train-target-folds/train_target_fold_3.csv
/kaggle/input/alfa-bank-train-target-folds/train_target_fold_5.csv
/kaggle/input/alfa-bank-train-target-folds/val_target_fold_7.csv
/kaggle/input/alfa-bank-train-target-folds/train_target_fold_4.csv
/kaggle/input/alfa-bank-train-target-folds/val_target_fold_1.csv
/kaggle/input/alfa-bank-train-target-folds/val_target_fold_5.csv
/kaggle/input/alfa-bank-train-target-folds/train_target_fold_1

### 0. Imports and requirements

In [2]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import sys
import pickle
import numpy as np#
from tensorflow import keras#
from tensorflow.keras import layers as L
from tensorflow.keras import Model


from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import lightgbm as lgb 
from bayes_opt import BayesianOptimization
from itertools import combinations

#import tqdm
from tqdm.notebook import tqdm
import time
import warnings
warnings.filterwarnings("ignore")



os.environ["CUDA_VISIBLE_DEVICES"] = "0"
pd.set_option('display.max_columns', None)

# здесь лежат функции для обработки данных
sys.path.append('./')

In [3]:
import matplotlib.pyplot as plt
plt.style.use("seaborn-pastel")

### 1. Data Preprocessing

In [4]:
TRAIN_DATA_PATH = "/kaggle/input/alfa-bank-pd-credit-history/data_for_competition/train_data"
TEST_DATA_PATH = "/kaggle/input/alfa-bank-pd-credit-history/data_for_competition/test_data"

#TRAIN_TARGET_PATH = "/kaggle/input/alfa-bank-pd-credit-history/data_for_competition/train_target.csv"
TRAIN_1_TARGET_PATH = "/kaggle/input/alfa-bank-train-target-folds/train_target_fold_1.csv"
VAL_1_TARGET_PATH = "/kaggle/input/alfa-bank-train-target-folds/val_target_fold_1.csv"

In [5]:
import os
import pandas as pd
from typing import List


def read_parquet_dataset_from_local(path_to_dataset: str, start_from: int = 0, num_parts_to_read: int = 2, 
                                    columns: List[str] = None, verbose: bool = False) -> pd.DataFrame:
    """
    Читает ``num_parts_to_read`` партиций и преобразует их к pandas.DataFrame.

    Параметры:
    -----------
    path_to_dataset: str
        Путь до директории с партициями.
    start_from: int, default=0
        Номер партиции, с которой начать чтение.
    num_parts_to_read: int, default=2
        Число партиций, которые требуется прочитать.
    columns: List[str], default=None
        Список колонок, которые нужно прочитать из каждой партиции. Если None, то считываются все колонки.

    Возвращаемое значение:
    ----------------------
    frame: pandas.DataFrame
        Прочитанные партиции, преобразованные к pandas.DataFrame.
    """

    res = []
    start_from = max(0, start_from)
    # dictionory of format {partition number: partition filename}
    dataset_paths = {int(os.path.splitext(filename)[0].split("_")[-1]): os.path.join(path_to_dataset, filename)
                     for filename in os.listdir(path_to_dataset)}
    chunks = [dataset_paths[num] for num in sorted(dataset_paths.keys()) if num>=start_from][:num_parts_to_read]
    
    if verbose:
        print("Reading chunks:", *chunks, sep="\n")
    for chunk_path in tqdm(chunks, desc="Reading dataset with pandas"):
        chunk = pd.read_parquet(chunk_path, columns=columns)
        res.append(chunk)
    return pd.concat(res).reset_index(drop=True)

In [6]:
from typing import List
import numpy as np
import pickle
import torch


features = ["pre_since_opened", "pre_since_confirmed", "pre_pterm", "pre_fterm", "pre_till_pclose", "pre_till_fclose",
            "pre_loans_credit_limit", "pre_loans_next_pay_summ", "pre_loans_outstanding", "pre_loans_total_overdue",
            "pre_loans_max_overdue_sum", "pre_loans_credit_cost_rate",
            "pre_loans5", "pre_loans530", "pre_loans3060", "pre_loans6090", "pre_loans90",
            "is_zero_loans5", "is_zero_loans530", "is_zero_loans3060", "is_zero_loans6090", "is_zero_loans90",
            "pre_util", "pre_over2limit", "pre_maxover2limit", "is_zero_util", "is_zero_over2limit", "is_zero_maxover2limit",
            "enc_paym_0", "enc_paym_1", "enc_paym_2", "enc_paym_3", "enc_paym_4", "enc_paym_5", "enc_paym_6", "enc_paym_7", "enc_paym_8",
            "enc_paym_9", "enc_paym_10", "enc_paym_11", "enc_paym_12", "enc_paym_13", "enc_paym_14", "enc_paym_15", "enc_paym_16",
            "enc_paym_17", "enc_paym_18", "enc_paym_19", "enc_paym_20", "enc_paym_21", "enc_paym_22", "enc_paym_23", "enc_paym_24",
            "enc_loans_account_holder_type", "enc_loans_credit_status", "enc_loans_credit_type", "enc_loans_account_cur",
            "pclose_flag", "fclose_flag"]


def batches_generator(list_of_paths: List[str], batch_size: int = 32, shuffle: bool = False,
                      is_infinite: bool = False, verbose: bool = False, device: torch.device = None,
                      output_format: str = "torch", is_train: bool = True):
    """
    Создает батчи на вход рекуррентных нейронных сетей, реализованных на фреймворках tensorflow и pytorch.

    Параметры:
    -----------
    list_of_paths: List[str]
        Список путей до файлов с предобработанными последовательностями.
    batch_size: int, default=32
        Размер батча.
    shuffle: bool, default=False
        Перемешивать ли данные перед генерацией батчей.
    is_infinite: bool, default=False
        Должен ли генератор быть бесконечным.
    verbose: bool, default=False
        Печатать ли имя текущего обрабатываемого файла.
    device: torch.device, default=None
        Девайс, на который переместить данные при ``output_format``="torch". Игнорируется, если ``output_format``="tf".
    output_format: str, default="torch"
        Формат возвращаемых данных. Допустимые значения: "torch", "tf".
        Если "torch", то возвращает словарь, с ключами "id_", "features" и "label", если is_train=True,
        и содержащий идентификаторы заявок, признаки и тагрет соответственно.
        Признаки и таргет помещаются на девайс, указанный в ``device``.
        Если "tf", то возращает кортеж (признаки, таргет), если ``is_train``=True, и кортеж (признаки, идентификаторы заявок) иначе.
    is_train: bool, default=True
        Используется ли генератор для обучения модели или для инференса.

    Возвращаемое значение:
    ----------------------
    result: dict or tuple
        Выходной словарь или кортеж в зависимости от параметра ``output_format``.
    """
    if output_format not in ["torch", "tf"]:
        raise ValueError("Unknown format. Please choose one of the following formats: \"torch\", \"tf\"")

    while True:
        if shuffle:
            np.random.shuffle(list_of_paths)

        for path in list_of_paths:
            if verbose:
                print(f"Reading {path}")

            with open(path, "rb") as f:
                data = pickle.load(f)

            ids, padded_sequences, targets = data["id"], data["padded_sequences"], data["target"]
            indices = np.arange(len(ids))
            if shuffle:
                np.random.shuffle(indices)
                ids = ids[indices]
                padded_sequences = padded_sequences[indices]
                if is_train:
                    targets = targets[indices]

            for idx in range(len(ids)):
                bucket_ids = ids[idx]
                bucket = padded_sequences[idx]
                if is_train:
                    bucket_targets = targets[idx]

                for jdx in range(0, len(bucket), batch_size):
                    batch_ids = bucket_ids[jdx: jdx + batch_size]
                    batch_sequences = bucket[jdx: jdx + batch_size]
                    if is_train:
                        batch_targets = bucket_targets[jdx: jdx + batch_size]

                    if output_format == "tf":
                        batch_sequences = [batch_sequences[:, i] for i in range(len(features))]

                        if is_train:
                            yield batch_sequences, batch_targets
                        else:
                            yield batch_sequences, batch_ids
                    else:
                        batch_sequences = [torch.LongTensor(batch_sequences[:, i]).to(device) for i in range(len(features))]
                        if is_train:
                            yield dict(id_=batch_ids,
                                       features=batch_sequences,
                                       label=torch.LongTensor(batch_targets).to(device))
                        else:
                            yield dict(id_=batch_ids,
                                       features=batch_sequences)
        if not is_infinite:
            break

In [7]:
from typing import Dict
import numpy as np
import pandas as pd
import pickle



def pad_sequence(array: np.ndarray, max_len: int) -> np.ndarray:
    """
    Принимает на вход массив массивов ``array`` и производит padding каждого вложенного массива до ``max_len``.

    Параметры:
    -----------
    array: numpy.ndarray
        Входной массив массивов.
    max_len: int
        Длина, до которой нужно сделать padding вложенных массивов.

    Возвращаемое значение:
    ----------------------
    output: numpy.ndarray
        Выходной массив.
    """
    if isinstance(max_len, float):
        print(max_len)
    output = np.zeros((len(features), max_len))
    output[:, :array.shape[1]] = array
    return output


def truncate(x, num_last_credits: int = 0):
    return pd.Series({"sequences": x.values.transpose()[:, -num_last_credits:]})


def transform_credits_to_sequences(credits_frame: pd.DataFrame,
                                   num_last_credits: int = 0) -> pd.DataFrame:
    """
    Принимает pandas.DataFrame с записями кредитных историй клиентов, сортирует кредиты по клиентам
    (внутри клиента сортирует кредиты от старых к новым), берет ``num_last_credits`` кредитов,
    возвращает новый pandas.DataFrame с двумя колонками: id и sequences.
    Каждое значение в столбце sequences - это массив массивов.
    Каждый вложенный массив - значение одного признака во всех кредитах клиента.
    Всего признаков len(features), поэтому будет len(features) массивов.

    Параметры:
    -----------
    credits_frame: pandas.DataFrame
        Датафрейм с записями кредитных историй клиентов.
    num_last_credits: int, default=0
         Количество кредитов клиента, которые будут включены в выходные данные. Если 0, то берутся все кредиты.

    Возвращаемое значение:
    ----------------------
    output: pandas.DataFrame
        Выходной датафрейм с двумя столбцами: "id", "sequences".
    """
    return credits_frame \
        .sort_values(["id", "rn"]) \
        .groupby(["id"])[features] \
        .apply(lambda x: truncate(x, num_last_credits=num_last_credits)) \
        .reset_index()


def create_padded_buckets(frame_of_sequences: pd.DataFrame, bucket_info: Dict[int, int],
                          save_to_file_path: str = None, has_target: bool = True):
    """
    Реализует Sequence Bucketing технику для обучения рекуррентных нейронных сетей.
    Принимает на вход датафрейм ``frame_of_sequences`` с двумя столбцами: "id", "sequences"
    (результат работы функции transform_credits_to_sequences),
    словарь ``bucket_info``, где для последовательности каждой длины указано, до какой максимальной длины нужно делать
    padding, группирует кредиты по бакетам (на основе длины), производит padding нулями и сохраняет результат
    в pickle файл, если требуется.

    Параметры:
    -----------
    frame_of_sequences: pandas.DataFrame
        Входной датафрейм с двумя столбцами "id", "sequences" (результат работы функции transform_credits_to_sequences).
    bucket_info: Dict[int, int]
        Cловарь, где для последовательности каждой длины указано, до какой максимальной длины нужно делать padding.
    save_to_file_path: str, default=None
        Опциональный путь до файла, куда нужно сохранить результат. Если None, то сохранение не требуется.
    has_target: bool, deafult=True
        Флаг, есть ли в frame_of_sequences целевая переменная или нет. Если есть, то она также будет записана в выходной словарь.

    Возвращаемое значение:
    ----------------------
    dict_result: dict
        Выходной словарь со ключами:  "id", "padded_sequences", "target".
    """
    frame_of_sequences["sequence_length"] = frame_of_sequences["sequences"].apply(lambda x: len(x[1]))
    frame_of_sequences["bucket_idx"] = frame_of_sequences["sequence_length"].map(bucket_info)
    padded_seq = []
    targets = []
    ids = []

    for size, bucket in tqdm(frame_of_sequences.groupby("bucket_idx"), desc="Extracting buckets"):
        padded_sequences = bucket["sequences"].apply(lambda x: pad_sequence(x, size)).values
        padded_seq.append(np.stack(padded_sequences, axis=0))

        if has_target:
            targets.append(bucket["flag"].values)

        ids.append(bucket["id"].values)

    frame_of_sequences.drop(columns=["bucket_idx"], inplace=True)

    dict_result = {
        "id": np.array(ids, dtype=np.object),
        "padded_sequences": np.array(padded_seq, dtype=np.object),
        "target": np.array(targets, dtype=np.object) if targets else []
    }

    if save_to_file_path:
        with open(save_to_file_path, "wb") as f:
            pickle.dump(dict_result, f)
    return dict_result

* Один из аргументов в функции `dataset_preprocessing_utils.create_padded_buckets` &ndash; `bucket_info` &ndash; словарь, где для конкретной длины последовательности указано до какой длины нужно делать паддинг. Для данного бэйзлайна возьмем простое разбиение на 43 бакета: 
| Длина последовательности | Длина после паддинга |
| :-: | :-: 
| 1 &ndash; 40 | без изменений |
| 41 &ndash; 45 | 45 |
| 46 &ndash; 50 | 50 |
| 51 &ndash; 58 | 58 |

In [10]:
keys_ = list(range(1, 59)) 
lens_ = list(range(1, 41)) + [45] * 5 + [50] * 5 + [58] * 8
bucket_info = dict(zip(keys_, lens_))
bucket_info

{1: 1,
 2: 2,
 3: 3,
 4: 4,
 5: 5,
 6: 6,
 7: 7,
 8: 8,
 9: 9,
 10: 10,
 11: 11,
 12: 12,
 13: 13,
 14: 14,
 15: 15,
 16: 16,
 17: 17,
 18: 18,
 19: 19,
 20: 20,
 21: 21,
 22: 22,
 23: 23,
 24: 24,
 25: 25,
 26: 26,
 27: 27,
 28: 28,
 29: 29,
 30: 30,
 31: 31,
 32: 32,
 33: 33,
 34: 34,
 35: 35,
 36: 36,
 37: 37,
 38: 38,
 39: 39,
 40: 40,
 41: 45,
 42: 45,
 43: 45,
 44: 45,
 45: 45,
 46: 50,
 47: 50,
 48: 50,
 49: 50,
 50: 50,
 51: 58,
 52: 58,
 53: 58,
 54: 58,
 55: 58,
 56: 58,
 57: 58,
 58: 58}

* Так же рассмотрим уникальные значения признаков

* Поскольку паддинг будет производиться нулями, а категориальные признаки закодированы, начиная с 0, перед паддингом будем сдвигать все значения на 1.

* Вся описанная выше предобработка данных реализована в виде функции `create_buckets_from_credits`:

In [11]:
def create_buckets_from_credits(path_to_dataset, bucket_info, save_to_path, frame_with_ids = None, 
                                num_parts_to_preprocess_at_once: int = 1, 
                                num_parts_total=50, has_target=False):
    block = 0
    for step in tqdm(range(0, num_parts_total, num_parts_to_preprocess_at_once),
                     desc="Preparing credit data"):
        credits_frame = read_parquet_dataset_from_local(path_to_dataset, step, num_parts_to_preprocess_at_once, verbose=True)
        credits_frame.loc[:, features] += 1       
        seq = transform_credits_to_sequences(credits_frame)
        print("Transforming credits to sequences is done.")
        
        if frame_with_ids is not None:
            seq = seq.merge(frame_with_ids, on="id")

        block_as_str = str(block)
        if len(block_as_str) == 1:
            block_as_str = "00" + block_as_str
        else:
            block_as_str = "0" + block_as_str
            
        processed_fragment =  create_padded_buckets(seq, bucket_info=bucket_info, has_target=has_target, 
                                                    save_to_file_path=os.path.join(save_to_path, 
                                                                                   f"processed_chunk_{block_as_str}.pkl"))
        block += 1

* Обучение и валидация будут происходить на семи фолдах.

In [14]:
TRAIN_BUCKETS_PATH_1 = "/kaggle/working/"

TRAIN_BUCKETS_PATH_1 = "/kaggle/working/train_buckets_fold_1"
!rm -rf $TRAIN_BUCKETS_PATH_1
!mkdir $TRAIN_BUCKETS_PATH_1

In [ ]:
VAL_BUCKETS_PATH_1 = "/kaggle/working/"

VAL_BUCKETS_PATH_1 = "/kaggle/working/val_buckets_fold_1"
!rm -rf $VAL_BUCKETS_PATH_1
!mkdir $VAL_BUCKETS_PATH_1

In [ ]:
TEST_BUCKETS_PATH = "/kaggle/working/"

TEST_BUCKETS_PATH = "/kaggle/working/test_buckets_rnn"
!rm -rf $TEST_BUCKETS_PATH
!mkdir $TEST_BUCKETS_PATH

In [15]:
train = pd.read_csv("/kaggle/input/alfa-bank-train-target-folds/train_target_fold_1.csv")
train

,id,flag
0,0,0
1,1,0
2,2,0
3,4,0
4,5,0
...,...,...
2571423,2999993,0
2571424,2999994,0
2571425,2999996,0
2571426,2999997,0


In [16]:
train.index = list(train['id'])

In [17]:
train

,id,flag
0,0,0
1,1,0
2,2,0
4,4,0
5,5,0
...,...,...
2999993,2999993,0
2999994,2999994,0
2999996,2999996,0
2999997,2999997,0


In [18]:
val = pd.read_csv("/kaggle/input/alfa-bank-train-target-folds/val_target_fold_1.csv")
val

,id,flag
0,3,0
1,8,0
2,9,0
3,13,0
4,19,0
...,...,...
428567,2999986,0
428568,2999988,0
428569,2999989,0
428570,2999995,0


In [19]:
val.index = list(val['id'])

In [20]:
val

,id,flag
3,3,0
8,8,0
9,9,0
13,13,0
19,19,0
...,...,...
2999986,2999986,0
2999988,2999988,0
2999989,2999989,0
2999995,2999995,0


In [21]:
create_buckets_from_credits(TRAIN_DATA_PATH,
                               bucket_info=bucket_info,
                               save_to_path=TRAIN_BUCKETS_PATH_1,
                               frame_with_ids=train,
                               num_parts_to_preprocess_at_once=4,
                               num_parts_total=12, has_target=True)

dataset_train = sorted([os.path.join(TRAIN_BUCKETS_PATH_1, x) for x in \
                        os.listdir(TRAIN_BUCKETS_PATH_1)])

Preparing credit data:   0%|          | 0/3 [00:00<?, ?it/s]

Reading chunks:
/kaggle/input/alfa-bank-pd-credit-history/data_for_competition/train_data/train_data_0.pq
/kaggle/input/alfa-bank-pd-credit-history/data_for_competition/train_data/train_data_1.pq
/kaggle/input/alfa-bank-pd-credit-history/data_for_competition/train_data/train_data_2.pq
/kaggle/input/alfa-bank-pd-credit-history/data_for_competition/train_data/train_data_3.pq


Reading dataset with pandas:   0%|          | 0/4 [00:00<?, ?it/s]

Transforming credits to sequences is done.


Extracting buckets:   0%|          | 0/43 [00:00<?, ?it/s]

...

Extracting buckets:   0%|          | 0/43 [00:00<?, ?it/s]

Reading chunks:
/kaggle/input/alfa-bank-pd-credit-history/data_for_competition/train_data/train_data_8.pq
/kaggle/input/alfa-bank-pd-credit-history/data_for_competition/train_data/train_data_9.pq
/kaggle/input/alfa-bank-pd-credit-history/data_for_competition/train_data/train_data_10.pq
/kaggle/input/alfa-bank-pd-credit-history/data_for_competition/train_data/train_data_11.pq


Reading dataset with pandas:   0%|          | 0/4 [00:00<?, ?it/s]

Transforming credits to sequences is done.


Extracting buckets:   0%|          | 0/43 [00:00<?, ?it/s]

In [22]:
dataset_train

['/kaggle/working/.virtual_documents',
 '/kaggle/working/__notebook_source__.ipynb',
 '/kaggle/working/processed_chunk_000.pkl',
 '/kaggle/working/processed_chunk_001.pkl',
 '/kaggle/working/processed_chunk_002.pkl']

In [25]:
create_buckets_from_credits(TEST_DATA_PATH,
                            bucket_info=bucket_info,
                            save_to_path=TEST_BUCKETS_PATH, num_parts_to_preprocess_at_once=2,
                            num_parts_total=2)

dataset_test = sorted([os.path.join(TEST_BUCKETS_PATH, x) for x in os.listdir(TEST_BUCKETS_PATH)])
dataset_test

Preparing credit data:   0%|          | 0/1 [00:00<?, ?it/s]

Reading chunks:
/kaggle/input/alfa-bank-pd-credit-history/data_for_competition/test_data/test_data_0.pq
/kaggle/input/alfa-bank-pd-credit-history/data_for_competition/test_data/test_data_1.pq


Reading dataset with pandas:   0%|          | 0/2 [00:00<?, ?it/s]

Transforming credits to sequences is done.


Extracting buckets:   0%|          | 0/43 [00:00<?, ?it/s]

['/kaggle/working/test_buckets_rnn/processed_chunk_000.pkl']

In [ ]:
%whos DataFrame